In [ ]:
!pip install openai

In [ ]:
!pip install faiss-cpu langchain openai tiktoken

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


https://docs.google.com/document/d/1f6zuyqT5TmYdybnPr13GRmp_iQ4l5MYmlYNni_HpuRk/edit?usp=sharing

In [ ]:
loader = TextLoader("/content/Артикулы.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=50, chunk_overlap=0)
docs = text_splitter.split_documents(documents)



In [ ]:
# всего получилось чанков:
len(docs)

61

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(docs, embeddings)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
db.save_local("/content/drive/My Drive/faiss_index")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import requests
import openai
from langchain.docstore.document import Document

In [ ]:
def load_document_text(url: str) -> str:
    # функция для загрузки документа по ссылке из гугл док
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

def answer_index(system, topic, search_index, temp=0, verbose=0):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=10)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp
    )
    if verbose: print('\n ===========================================: ')
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ


In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system = 'в ответе никогда не указывай номер отрывка документа, найди в базе один самый похожий артикула по цифрам и буквам и их порядку с тем что введет тебе пользователь'

In [ ]:
# в таблице RB 7091 8142 53*19 140
# проверяем RB 7891 0142 53*19 140
topic= "RB 7891 0142 53*19 130"

In [ ]:
ans=answer_index(system, topic, db, temp=0.1, verbose=0)
ans

'Самый похожий артикул в базе данных на ваш запрос "RB 7891 0142 53*19 130" - это отрывок документа №3: "RB 7091 8142 53*19 140".'

In [ ]:
# в таблице LN50025 C3 50*19 143
# проверяем LN68025 53 50o19 143
topic= "LN68025 C3 50o19 143"
ans=answer_index(system, topic, db, temp=0.5, verbose=0)
ans

'Похожий артикул из базы данных: LN50025 C3 50*19 143'

In [ ]:
# в таблице GU 1855 BRN 54*17 140
# проверяем 60 7855 BRN 54о17 130
topic= "60 7855 BRN 54о17 130"
ans=answer_index(system, topic, db, temp=0.5, verbose=0)
ans

'Самый похожий артикул на введенный клиентом запрос "60 7855 BRN 54о17 130" это "GU 1855 BRN 54*17 140" из отрывка документа №1.'

In [ ]:
# в таблице VYF120 C02 55*18 140
# проверяем YKL120 CoG 54о17 130
topic= "YKL120 CoG 54о17 130"
ans=answer_index(system, topic, db, temp=0.5, verbose=0)
ans

'Самый похожий артикул на введенный клиентом запрос "YKL120 CoG 54о17 130" - это "VYF120 C02 55*18 140" из отрывка документа №1.'